In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [3]:
from lightfm import LightFM

In [32]:
# Quick Data Prep

In [4]:
raw = pd.read_csv('data/candy.csv')
raw.head(5)

,candy,max,zekry,jonathan,mousa,blair,vishal,megan,sara,lucas,cody,minh
0,100 Grand,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3 Musketeers,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN
2,One dime,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,One quarter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Air Heads,4.0,NaN,8.0,NaN,5.0,5.0,6.0,5.0,NaN,NaN,NaN


In [5]:
df = pd.melt(raw, id_vars='candy', var_name='user', value_name='rating')
df = df.dropna()
df.head(5)

,candy,user,rating
4,Air Heads,max,4.0
8,Candy Corn,max,1.0
10,Charleston Chew,max,3.0
12,Chiclets,max,4.0
13,Dots,max,6.0


In [33]:
# The Magic

In [37]:
class InteractionMachine:
    
    def __init__(self, df, ratings, users, items):
        self._ratings = np.array(df[ratings])
        self._users = np.array(df[users])
        self._items = np.array(df[items])
        # heavy lifting encoders
        self.user_encoder = LabelEncoder()
        self.item_encoder = LabelEncoder()
        # preparation for the csr matrix
        u = self.user_encoder.fit_transform(self._users)
        i = self.item_encoder.fit_transform(self._items)
        uu = len(np.unique(u))
        ui = len(np.unique(i))
        # the good stuff
        self.interactions = csr_matrix((self._ratings, (u, i)), shape=(uu, ui))
    
    def get_users(self, encoded=False):
        users = np.unique(self._users)
        if encoded:
            users = self.user_encoder.transform(users)
        return users
    
    def get_items(self, encoded=False):
        items = np.unique(self._items)
        if encoded:
            items = self.item_encoder.transform(items)
        return items

In [38]:
im = InteractionMachine(df, 'rating', 'user', 'candy')
im.interactions

<9x66 sparse matrix of type '<class 'numpy.float64'>'
	with 310 stored elements in Compressed Sparse Row format>

In [39]:
im.get_items(encoded=True)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65])

In [41]:
im.get_users(encoded=False)

array(['blair', 'cody', 'jonathan', 'max', 'megan', 'mousa', 'sara',
       'vishal', 'zekry'], dtype=object)

In [42]:
pd.DataFrame(
    im.interactions.todense(),
    index = im.get_users(),
    columns = im.get_items()
)

,3 Musketeers,Air Heads,Almond Joy,Candy Corn,Charleston Chew,Chewey Lemonhead Fruit Mix,Chiclets,Dots,Fruit Chews,Fun Dip,...,Swedish Fish,Tootsie Pop,Tootsie Rolls,Twix,Twizzlers,Warheads,Welch's Fruit Snacks,Werther's Original Caramel,Whoppers,Wine gums
blair,0.0,5.0,3.0,2.0,0.0,0.0,2.0,0.0,0.0,7.0,...,7.0,5.0,6.0,9.0,7.0,6.0,3.0,4.0,0.0,7.0
cody,7.0,0.0,0.0,0.0,6.0,0.0,4.0,0.0,0.0,0.0,...,6.0,0.0,5.0,10.0,6.0,6.0,9.0,8.0,0.0,8.0
jonathan,8.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5.0,0.0,5.0,0.0,0.0,8.0,5.0,0.0,9.0
max,0.0,4.0,0.0,1.0,3.0,0.0,4.0,6.0,0.0,6.0,...,6.0,0.0,1.0,7.0,0.0,6.0,6.0,3.0,0.0,7.0
megan,0.0,6.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,9.0,...,0.0,1.0,1.0,4.0,6.0,6.0,5.0,9.0,9.0,8.0
mousa,0.0,0.0,0.0,8.0,0.0,8.0,0.0,0.0,8.0,0.0,...,0.0,0.0,0.0,9.0,9.0,0.0,0.0,0.0,0.0,0.0
sara,0.0,5.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,7.0,4.0,0.0,2.0,3.0,0.0,0.0
vishal,0.0,5.0,0.0,4.0,0.0,0.0,3.0,3.0,0.0,5.0,...,0.0,0.0,0.0,8.0,7.0,6.0,8.0,4.0,0.0,5.0
zekry,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,2.0,0.0,0.0,6.0,0.0,0.0,0.0,7.0,0.0,9.0


In [43]:
model = LightFM()

In [44]:
model.fit(im.interactions, epochs=20)

In [46]:
person = 'max'
user_id = im.user_encoder.transform([person])[0]
preds = model.predict(user_id, im.get_items(encoded=True))

In [49]:
pd.DataFrame({
    'candy': im.get_items(),
    'rating': preds
}).sort_values('rating')

,candy,rating
19,Jawbusters,1.344739
58,Tootsie Rolls,1.584914
57,Tootsie Pop,1.747719
44,Ring pop,1.824791
35,Nik L Nip,1.880275
14,Haribo Twin Snakes,1.888598
64,Whoppers,1.901769
29,Milky Way Simply Caramel,1.908305
28,Milky Way Midnight,1.912672
30,Mounds,1.914338
